# Proposal Calculations

This is the program to calculate proposals.

## Philosophy

The idea is to use excel only for data input, shareing with others and for presentation purpose. Excel is still needed to be used as it is what most people are familiar with, both internal and external, though it is not ideal. Therefore, Excel use will be minimize as much as possible. All the heavy lifting and intelligence will be handed over to the programming.

## Input Excel File Specification


- The System name must ends with "System", case insensitive. There must be nothing in 'No' field.
- There must be a number (any number) for Item in 'No' field. Appropriate number will be replaced by the program if any number is present.
- If there is cost, there will be a Subitem Number (SN) assigned by the program. In this sense, there is no real care for Sub-No field, just that it may help one thinks in initial proposal writing.

Format titles

- System: Ends with "System" (case insensitive) there is not anything in 'No' field.
- Item: Any item that has a number in 'No' field and description is not empty.
- Subitem: Any item where 'Cost' field is zero and above and 'Qty' > 1.
- Title: Optionally use to title 'Item'. Must not end the spelling with 'System' as the program will confuse this with 'System'.
- Subtitle: Optionally use to title 'Subitem'. Check that it itself is not an 'Subitem', it does not start with a '•' and there exists a 'Subitem' immediately. '~' or '-' at the start of line will be replaced by '•'.
- Comment: Description must start with '***'.

Breakdown Price or Lump Sum

- Lump Sum is applicable only for 'Item' where it is indicated as 1 lot. Lot can only be 1 lot. More than one causes error. If one lot, there will be lumpsum price in Selling Price.

### Fields in Excel

- Scope: 'Option' for optional items. If subitem cost is '0' or sell at '0' price, it is to be marked as 'Inclusive'.
- Category:

### In Excel

- To use data validation for critical fields.
- Catetory field to differentiate whether it is product or service. This will affect the risk factor. This may extend to license and other categories.
- Discount to be in percentage, i.e. to already have been divided by 100. If not to include formula to introduce this change. Basic rule is that the number we talk in terms of percentages must be presented being already divided by 100.
- In excel sheet, it is needed to add in "Category", "System" and "Fixed Unit Price" columns
- There must be a blank line after the system title

## Things to implement

- Try statement for iterating through df as some values are nan.
- Look for non common word in 'Description' before checking againt common English words. The code will take time, if every word is checked against nltk dictionary so better to do this loop less.

## Things to implement

- [x] Try statement for iterating through dataframe as some values are nan.
- [x] Build efficient acronyms file. Look for non common word in 'Description' before checking againt common English words. The code will take time, if every word is checked against nltk dictionary so better to do this loop less.
- [ ] Implement a function to change the text in particular cell in DataFrame

In [2]:
# Custom functions to set the right font cases

def set_case(text, case='title'):
    if case == 'title':
        text = text.title()
        for word in acronyms:
            pattern = r'\b' + re.escape(word.title()) + r'\b'
            text = re.sub(pattern, word, text)
    elif case == 'sentence':
        # Handle subitems
        if re.match('^• ', text):
            text = text[2:].capitalize()
            text = '• ' + text
        else:
            text = text.capitalize()
        for word in acronyms:
            pattern = r'\b' + re.escape(word.lower()) + r'\b'
            text = re.sub(pattern, word, text)
            pattern = r'\b' + re.escape(word.title()) + r'\b'
            text = re.sub(pattern, word, text)
    elif case == 'upper':
        text = text.upper()
        for word in acronyms:
            pattern = r'\b' + re.escape(word.upper()) + r'\b'
            text = re.sub(pattern, word, text)
    elif case == 'all_upper':
        text = text.upper()
    return text

# Get system name from file name. This will be used in 'System' field.
def get_system_name (file_name):
    file_name = file_name[file_name.rfind(r' - '):]
    file_name = file_name[:file_name.rfind('R0')]
    file_name = file_name.strip('- ')
    system_name = file_name.strip().upper()
    return system_name

# Function to set the value of the cell on index and value
def set_cell(idx, value, column='Description'):
    df.at[idx, column] = value

# Function to show the value of the cell on index
def show_cell(idx, column='Description'):
    print(str(df.at[idx, column]))

# Collection of format functions to be applied together
def set_format(text):
    text = set_single_space(text)
    text = set_comma_space(text)
    text = set_x(text)
    return text

# Strip 2 or more spaces
def set_single_space (text):
    return re.sub(' {2,}', ' ',  text)

# Set space after comma
def set_comma_space(text):
    x = re.compile(',\w+')
    if x.search(text):
        substring = re.findall(',\w+', text)
        for word in substring:
            text = re.sub(word, ', ' + word[1:], text)
        return text

# Function to replace description such as 1x, 20x, 10X , x1, x20, X20 into 1 x, 20 x, 10 x, x 1, x 20, X 10 etc.
def set_x(text):
    # For cases such as 20x, 30X
    x = re.compile('(\d+x|\d+X)')
    if x.search(text):
        substring = re.findall('(\d+x|\d+X)', text)
        for word in substring:
            text = re.sub(word, (word[:-1] + ' x'), text)
    # For cases such as x20, X30
    x = re.compile('(x\d+|X\d+)')
    if x.search(text):
        substring = re.findall('(x\d+|X\d+)', text)
        for word in substring:
            text = re.sub(word, ('x ' + word[1:]), text)
    # For cases such as 20 X, 30 X
    x = re.compile('(\d+ X)')
    if x.search(text):
        substring = re.findall('(\d+ X)', text)
        for word in substring:
            text = re.sub(word, (word[:-1] + 'x'), text)
    # For cases such as X 20, X 30
    x = re.compile('(X \d+)')
    if x.search(text):
        substring = re.findall('(X \d+)', text)
        for word in substring:
            text = re.sub(word, ('x' + word[1:]), text)
    return text

In [41]:
# Need to read multiple Excel files from selected directory and combine the data

path = '/Users/infowizard/Downloads/Project'
# path = './Temp'
files = os.listdir(path)
files.sort() #Sort file names. List type sorts in place.

# Need to initialize df with proper columns
d = {'No' : [] , 'Sub-No': [], 'Description' : [], 'Qty' : [] , 'Unit' : [] , 'Cur' : [] , 'Cost' : [] ,
     'Discount' : [] , 'Scope' : [] , 'Supplier' : [] , 'Maker' : [] , 'Model' : [] , 'System' : [] ,
     'Category' : [] , 'Fixed Unit Price' : [] , 'Remark' : []}
df = pd.DataFrame(d)

for filename in files:
    if filename[-4:] == 'xlsx' and filename[0] != '~':
        with open(path + '/' + filename, 'rb') as file:    # Need to open as 'rb' as it is throwing utf-8 error.
            print(filename)
            temp = pd.read_excel(file)
            if not 'Fixed Unit Price' in temp.columns:
                temp['Fixed Unit Price'] = np.nan
            if not 'System' in temp.columns:
                temp['System'] = np.nan
            if not 'Category' in temp.columns:
                temp['Category'] = np.nan

            # temp['System'] = temp['System'].where(pd.isnull(temp['Qty']), get_system_name(filename))
            # temp['Category'] = temp['Category'].where(pd.isnull(temp['Qty']), 'Product')

            for idx in temp.index:
                if pd.notna(temp.loc[idx, 'Qty']):
                    temp.loc[idx, 'System'] = get_system_name(filename)
                    if pd.notna(temp.loc[idx, 'Cost']):
                        temp.loc[idx, 'Category'] = 'Product'

            df = pd.concat([df, temp], ignore_index=True)
        file.close()

# Set 'Scope' to title case
try:
    df['Scope'] = df['Scope'].str.title()
except Exception as e:
    print(str(e))

01 J12273 MODEC - BM-C-33 FPSO - GMDSS R0.xlsx
02 J12273 MODEC - BM-C-33 FPSO - OPERATION SUPPORT R0.xlsx
03 J12273 MODEC - BM-C-33 FPSO -  UHF R0.xlsx
04 J12273 MODEC - BM-C-33 FPSO - VHF R0.xlsx
05 J12273 MODEC - BM-C-33 FPSO - CRANE R0.xlsx
06J12273 MODEC - BM-C-33 FPSO - NAV R0.xlsx
07 J12273 MODEC - BM-C-33 FPSO - COMM SPARES R0.xlsx
08 J12273 MODEC - BM-C-33 FPSO - SPARES R0.xlsx
09 J12273 MODEC - BM-C-33 FPSO - ENGINEERING SERVICES R0.xlsx


In [42]:
# Set engineering services to "Service" category
for idx in df.index:
    if str(df.loc[idx, 'System']).lower() == 'engineering services':
        df.loc[idx, 'Category'] = 'Service'

In [273]:
# Clean data

# Sometimes, copy and paste ends up with extra spaces.
df['Description'] = df['Description'].str.strip()

# Put bullet point for Sub-subitem preceded by '-' or '~'.
df['Description'] = df['Description'].str.replace('^(-|~)', '•', regex=True)

# Put bullet point for Sub-subitem preceded by a single * followed by space.
df['Description'] = df['Description'].str.replace('^[*?]\s', ' • ', regex=True)

# I do not like ';' at the end of line. Use hyphen instead.
df['Description'] = df['Description'].str.replace(';$', ':', regex=True)

# I do not like description like 1x without proper spacing. There must be one space in between.
for idx in df.index:
    try:
        df.loc[idx, 'Description'] = set_x(df.loc[idx, 'Description'])
    except Exception as e:
        pass

In [274]:
# Strip multiple spaces into single space
df['Description'] = df['Description'].str.replace('[  ]{2,}', ' ', regex=True)

In [275]:
# If there is already acronyms file, we can read in from there. Otherwise, we need to get it from the data.
# If you want to reset, delete or rename acronyms.txt file.

# Add word, which must always remain the same case
to_add = ['Ka', 'Ex', 'Zone', 'of', 'Catalyst', 'SNTC-8X5xNBD', 'LED', 'LEDs', 'mP240MT', 'ATS', 'PoE', 'c/w',
          'AC', 'ADE', 'PA', 'Cat6A', 'SART', 'IE', 'AIS', 'ONVIF', 'LAN', 'POB', 'UPS', 'ESD', 'INS', 'Mini-C',
          'FAT', 'DAS', 'PRMS']

# Words to be removed as these can be treated as normal English words
to_remove = ['Archiving', 'Cassette', 'WORKSTATIONS', 'services']

try:
    file = open('acronyms.txt')
    data = file.read()
    acronyms = data.split('\n')
    print('Using "acronyms.txt"')
except:
    print('File "acronyms.txt" does not exists. Collecting acronyms from data...')
    collection = set()
    for idx in range(len(df)):
        # Need to convert to string to check for nan values.
        try:
            item = (df.loc[idx, 'Description']).split(' ')
            for word in item:
                # Check is item is only a single character 'a' or 'A' and not all digit
                if re.match('^[aA?]$', word) or not word.isdigit():
                    # Remove non-alphanumeric characters except and degree symbol
                    word = re.sub('[^\w°™]$', '', word)
                    # Remove parenthesis and '=' and m-dash.
                    word = re.sub('[()=–\s]', '', word)
                    # Remove empty words and words without any caracter (i.e. digits and symbols)
                    if len(word) > 1 and re.sub('[^a-zA-Z]*', '', word):
                        collection.add(word)
        except Exception as e:
            # print(str(e))
            pass

    # Do the first pass to filter out common English words. It does not catch plurals.
    acronyms_unfiltered = set()
    for word in collection:
        if word.lower() not in dictionary.words():
            acronyms_unfiltered.add(word)

    # Do the second pass to filter out plural (stem) English words.
    acronyms = acronyms_unfiltered.copy()
    stemmer = porter.PorterStemmer()
    for word in acronyms_unfiltered:
        if stemmer.stem(word) in dictionary.words():
            acronyms.remove(word)

    acronyms = list(acronyms)
    acronyms.sort(reverse=True)
    # Write file out for further reference.
    with open('acronyms.txt', 'w+') as f:
        for word in acronyms:
            f.write("%s\n" %word)
    f.close()
    print('Done collecting acronyms.')
finally:
    # Doing this so that upper case of the same spelling is preferred. E.g. prefers 'AC' to 'ac'.
    acronyms.sort(reverse=True)
    try:
        for word in to_remove:
            acronyms.remove(word)
    except Exception as e:
        print(str(e))
        # pass
    acronyms = acronyms + to_add

File "acronyms.txt" does not exists. Collecting acronyms from data...
Done collecting acronyms.
list.remove(x): x not in list


In [276]:
# Differentiation of line item types, numbering, setting case and format column for further processing in excel.

# Creat empty format field, which gets reset everytime the code is run.
# This help with renaming items which ends with 'System' that are not System title
# This way, once data is imported, the Excel file needs not be open manually to change.
df['Format'] = ''

# The System name must ends with "System", case-insensitive. There must be nothing in 'No' field.
count = 1
for idx in df.index:
    if not pd.notna(df.loc[idx, 'No']):
        if str(df.loc[idx, 'Description']).lower().endswith('system') \
            and not str(df.loc[idx, 'Description']).startswith('•'):
            df.at[idx, 'Description'] = df.at[idx, 'Description'].upper()
            df.at[idx, 'Format'] = 'System'
            print(count, ': ', idx, ': ', df.loc[idx, 'Description'])
            count += 1


In [ ]:
# setcell(2685, 'DPS I4 HIGH PERFORMANCE GNSS')
# setcell(2719, 'TRIMBLE HYDROPRO SOFTWARE FOR COMPUTER')
# showcell(2719)

In [277]:
# Items:
# Set case and number for items. Any item that has a number in 'No' field and description is not empty.

count = 1
for idx in df.index:
    if pd.notna(df.loc[idx, 'No']) and pd.notna(df.loc[idx, 'Description']):
        try:
            # See if the value can be converted to integer.
            # Notice float is used as passing string in float format causes error.
            number = int(float(str(df.at[idx, 'No'])))
            df.at[idx, 'Description'] = set_case(str(df.at[idx, 'Description']))
            df.at[idx, 'Format'] = 'Item'
            df.at[idx, 'No'] = count
            df.at[idx, 'Sub-No'] = np.nan
            count += 1
        except Exception as e:
            # print(str(e))
            pass

In [278]:
# Lineitems:
# Numbering of the lineitems.
# Lineitems must have values in Description, Qty and Cost.

# Reset Sub-No row as there may be some unclean data by manually numbering rows.
df['Sub-No'] = np.nan
count = 1
for idx in df.index:
    if pd.notna(df.loc[idx, 'No']):
        count = 1    # Reset count

    if pd.notna(df.loc[idx, 'Description']) & pd.notna(df.loc[idx, 'Qty']) & pd.notna(df.loc[idx, 'Cost']):
        df.at[idx, 'Sub-No'] = count
        df.at[idx, 'Format'] = 'Lineitem'
        df.loc[idx, 'Description'] = set_case(df.loc[idx, 'Description'])
        count += 1

In [279]:
# Title:
#Optionally use to make some descriptions clear.

for idx in df.index:
    if not pd.notna(df.loc[idx, 'No']) \
        and not pd.notna(df.loc[idx, 'Sub-No']) \
        and pd.notna(df.loc[idx, 'Description']) \
        and not str(df.loc[idx, 'Description']).lower().endswith('system')\
        and not str(df.loc[idx, 'Description']).startswith('***'):
            # print(df.loc[idx, 'Description'])
            try:
                # Check for empty line above
                if not pd.notna(df.loc[idx - 1, 'Description']):
                    # print(df.loc[idx, 'Description'])
                    # Check if any of the two rows below is item.
                    if pd.notna(df.loc[idx + 1, 'No']):
                        print(df.loc[idx, 'Description'])
                        df.loc[idx, 'Format'] = 'Title'
                        df.loc[idx, 'Description'] = set_case(df.loc[idx, 'Description'])
                    elif pd.notna(df.loc[idx + 2, 'No']):
                        print(df.loc[idx, 'Description'])
                        df.loc[idx, 'Format'] = 'Title'
                        df.loc[idx, 'Description'] = set_case(df.loc[idx, 'Description'])
            except Exception as e:
                # print(str(e))
                pass

A. VSAT SYSTEM - O3B
A. VSAT SYSTEM - FBB
MECHANICAL COMPLETION EQUIPMENT
C. 2 YRS SPARES (OPTION)
ENGINEERING SERVICES


In [280]:
# Subtitle:
# Optionally use to title 'Subitem'. Check that it itself is not an 'Subitem',
# it does not start with a '•' and there exists a 'Subitem' immediately. '~' or '-' at
# the start of line will be replaced by '•'. If the 'Format' field already has value, ignore.

for idx in df.index:
    if not pd.notna(df.loc[idx, 'No']) and not pd.notna(df.loc[idx, 'Sub-No']) and \
    pd.notna(df.loc[idx, 'Description']):
        if str(df.loc[idx, 'Description'])[0] != '•':
            try:
                if pd.notna(df.loc[idx + 1, 'Sub-No']):
                    df.loc[idx, 'Format'] = 'Subtitle'
                    # Set title case
                    df.loc[idx, 'Description'] = set_case(df.loc[idx, 'Description'])
            except Exception as e:
                # print(str(e))
                pass

In [281]:
# The comment field, which starts with ***

for idx in df.index:
    if str(df.loc[idx, 'Description']).startswith('***'):
        try:
            # Remove preceding ***
            string = str(df.at[idx, 'Description'])[4:]
            # Split into sentences.
            sentences = []
            for sentence in re.split('(?<=[.!?])\s+', string):
                sentences.append(set_case(sentence, 'sentence'))
            comment = ' '.join(sentences)
            df.at[idx, 'Description'] = '*** ' + comment
            df.at[idx, 'Format'] = 'Comment'
        except Exception as e:
            # print(str(e))
            pass

In [282]:
# Set sentence case otherwise.

# Change to customize cases. This solves the most bothering problem that I have. Finally, huh!
check = ('System', 'Title', 'Item', 'Lineitem', 'Subtitle', 'Comment')
for idx in df.index:
    if df.loc[idx, 'Format'] not in check:
        try:
            df.loc[idx, 'Description'] = set_case(df.loc[idx, 'Description'], 'sentence')
        except Exception as e:
            pass

In [350]:
# Read the prepared file.
df = pd.read_excel('/Users/infowizard/Downloads/J12272 MODEC - BM-C-33 FPSO - VSAT SOURCE R0.xlsx')

In [268]:
d = {'No' : [] , 'Sub-No': [], 'Description' : [], 'Qty' : [] , 'Unit' : [] , 'Cur' : [] , 'Cost' : [] ,
     'Discount' : [] , 'Scope' : [] , 'Supplier' : [] , 'Maker' : [] , 'Model' : [] , 'System' : [] ,
     'Category' : [] , 'Fixed Unit Price' : [] , 'Remark' : []}
df1 = pd.DataFrame(d)

In [269]:
df = pd.concat([df, df1], ignore_index=True)

In [348]:
# Write the prepared file.
df.to_excel('/Users/infowizard/Downloads/J12272 MODEC - BM-C-33 FPSO - VSAT SOURCE R0.xlsx', index=False)

In [352]:
# Initialization

# Currency to quote in
quoted_currency = "USD"

# Set exchange rates: Monday 2022-11-14
currency_sgd_based = {"SGD" : 1.00,
                      "USD" : 1.45,
                      "EUR" : 1.43,
                      "GBP" : 1.67,
                      "DKK" : 0.1948,
                      "NOK" : 0.1408,
                      "CNY" : 0.2003,
                      "AUD" : 0.93

                     }

currency_usd_based = {"SGD" : 0.69,
                      "USD" : 1.00,
                      "EUR" : 1.00,
                      "GBP" : 1.15,
                      "DKK" : 0.1343,
                      "NOK" : 0.0971,
                      "CNY" : 0.1381,
                      "AUD" : 0.64
                     }

currency = {}

if quoted_currency == "SGD":
    currency = currency_sgd_based
else:
    currency = currency_usd_based

# Overhead cost defined. To be coverted to % in formula
default = 3.0
warranty = 3.0
freight = 5.0 # Inbound freight
special = 1.0

# Therefore, total risk escalation on the cost as percentage
escalation = (default + warranty + freight + special) / 100    # In percentage

# Risk factor based on category
risk_factor = {"Product" : escalation, "Service" : 0.0}

# Inherent risk in project
risk = 5.0 / 100    # In percentage

# Mark Up
recommended_mu = 3 / 100    # In percentage


In [353]:
# Calculate Cost

# Apply discount
# Be careful of the way discount is presented and adapt the formula accordingly. Whether to divide by 100 or not.
df['Discounted Cost'] = np.nan
for idx in df.index:
    try:
        df.loc[idx, 'Discounted Cost'] = (df.loc[idx, 'Cost'] * (1 - df.loc[idx, 'Discount']))
    except Exception as e:
        #print(str(e))
        pass

# Map function for risk escalation.

df['Escalation'] = df['Category'].map(risk_factor)

# Unit Cost after Escalation In Base Currency (UCE). Cost escalation for overhead, deliveries and other factors.
df['UCE'] = np.nan
for idx in df.index:
    try:
        df.loc[idx, 'UCE'] = (df.loc[idx, 'Discounted Cost'] * (1 + df.loc[idx, 'Escalation']))
    except Exception as e:
        # print(str(e))
        pass

# Unit cost after risk in base currency (UCR). Risk applies to all categories.
df['UCR'] = (df['UCE'] / (1 - risk))

# Map function for currency conversion
df['Rate'] = df['Cur'].map(currency)

# Calculate unit cost in currency to quote
df['Unit Cost'] = np.nan
for idx in df.index:
    try:
        df.loc[idx, 'Unit Cost'] = (df.loc[idx, 'UCR'] * (df.loc[idx, 'Rate']))
    except Exception as e:
        # print(str(e))
        pass

# Calculate subtotal cost. Skip if the item is option. For 'inclusive' the cost needs to be considered.
for idx in df.index:
    if str(df.loc[idx, 'Scope']).lower() not in ['option']:
        df.loc[idx, 'Subtotal Cost'] = df.loc[idx, 'Qty'] * df.loc[idx, 'Unit Cost']

In [354]:
# Calculate selling prices

# Recommended unit price based on set margin (RUP)
df['RUP'] = (df['Unit Cost'] / (1 - recommended_mu))

# Recommended subtotal price based on set margin (RSP)
df['RSP'] = (df['Qty'] * df['RUP'])

# Unit selling price at defautl margin
df['Unit Price'] = np.ceil(df['RUP'])

# Apply Fixed Unit Price if any. Especially useful for Engineering Services.
df['Unit Price'] = df['Fixed Unit Price'].where(df['Fixed Unit Price'] > 0, df['Unit Price'])

# Subtotal selling price.
df['Subtotal Price'] = np.nan    # Reset
for idx in df.index:
    if str(df.loc[idx, 'Scope']).lower() not in ['option', 'inclusive']:
        df.loc[idx, 'Subtotal Price'] = df.loc[idx, 'Qty'] * df.loc[idx, 'Unit Price']

# Profit
df['Profit'] = df['Subtotal Price'] - df['Subtotal Cost']

# Margin (Technically)
df['Margin'] = df['Profit'] / df['Subtotal Price']

In [355]:
# Fill the gap in unit prices
df['Unit Price LS'] = np.nan

for idx in df.index:
    # If it is lump sum
    if pd.notna(df.loc[idx, 'No']) \
        and pd.notna(df.loc[idx, 'Description']) \
        and str(df.loc[idx, 'Unit']).lower() == 'lot':
            try:
                if int(float(str(df.loc[idx, 'Qty']))) == 1:
                    while True:
                        # Look for subitems
                        if pd.notna(df.loc[idx + 1, 'Sub-No']):
                            if str(df.loc[idx + 1, 'Scope']).lower() in ['option', 'inclusive']:
                                df.loc[idx + 1, 'Unit Price LS'] = df.loc[idx + 1, 'Unit Price']
                        if pd.notna(df.loc[idx + 1, 'No']):
                            break
                        idx += 1
            except Exception as e:
                print(str(e))
    # If it is not lump sum
    elif pd.notna(df.loc[idx, 'No']) and pd.notna(df.loc[idx, 'Description']) \
        and not pd.notna(df.loc[idx, 'Qty']) and not pd.notna(df.loc[idx, 'Unit']):
        while True:
             # Look for subitems
            if pd.notna(df.loc[idx + 1, 'Sub-No']):
                df.loc[idx + 1, 'Unit Price LS'] = np.ceil(df.loc[idx + 1, 'Unit Price'])
                # print(df.loc[idx, 'Description'])
            if pd.notna(df.loc[idx + 1, 'No']):
                break
            # Reaches at the end
            if idx == len(df) - 2 :
                break
            idx += 1

473


In [356]:
# Come on, let us do the challenging part of doing lumpsum pricing
# Lump sum price is needed when it is an item, description is not empty,
# quantity is exactly 1, the unit is exactly lot

df['Subtotal Price LS'] = np.nan

for idx in df.index:
    # If it is lump sum
    if pd.notna(df.loc[idx, 'No']) \
        and pd.notna(df.loc[idx, 'Description']) \
        and str(df.loc[idx, 'Unit']).lower() == 'lot':
            try:
                if int(float(str(df.loc[idx, 'Qty']))) == 1:
                    row_idx = idx
                    # print(df.loc[idx, 'Description'])
                    # Look for subitems
                    subtotal = 0
                    while True:
                        # Look for subitems
                        if pd.notna(df.loc[idx + 1, 'Sub-No']):
                            if str(df.loc[idx + 1, 'Scope']).lower() not in ['option', 'inclusive']:
                                subsubtotal = np.ceil(df.loc[idx + 1, 'Qty'] * df.loc[idx + 1, 'Unit Price'])
                                #print(type(subsubtotal))
                                if not np.isnan(subsubtotal):
                                    subtotal += subsubtotal
                                # print(subtotal)
                        if pd.notna(df.loc[idx + 1, 'No']):
                            # print(df.loc[idx + 1, 'No'])
                            # Set Subtotal price
                            df.loc[row_idx, 'Unit Price LS'] = subtotal
                            df.loc[row_idx, 'Subtotal Price LS'] = subtotal
                            # print('Subtotal: ' + str(df.loc[row_idx, 'Subtotal Price LS']))
                            break
                        idx += 1
            except Exception as e:
                print(str(e))
    # If it is not lump sum
    elif pd.notna(df.loc[idx, 'No']) and pd.notna(df.loc[idx, 'Description']) \
        and not pd.notna(df.loc[idx, 'Qty']) and not pd.notna(df.loc[idx, 'Unit']):
        while True:
             # Look for subitems
            if pd.notna(df.loc[idx + 1, 'Sub-No']):
                if str(df.loc[idx + 1, 'Scope']).lower() not in ['option', 'inclusive']:
                    df.loc[idx + 1, 'Subtotal Price LS'] = np.ceil(df.loc[idx + 1, 'Qty'] * df.loc[idx + 1, 'Unit Price'])
            if pd.notna(df.loc[idx + 1, 'No']):
                break
            # Reaches at the end
            if idx == len(df) - 2 :
                break
            idx += 1


473


In [358]:
# Working on the excel file now
df.to_excel('/Users/infowizard/Downloads/J12272 MODEC - BM-C-33 FPSO - VSAT COSTING R0.xlsx', index=False)

In [357]:
# Getting summary data
summary_to_report = df.groupby(['System'])[['Subtotal Cost', 'Subtotal Price', 'Profit']].sum()
total_margin = (summary_to_report['Profit'].sum() /summary_to_report['Subtotal Price'].sum())
pd.set_option('display.float_format', lambda x: '%0.2f' % x)
summary = df.groupby(['System'])[['Subtotal Cost', 'Subtotal Price', 'Profit']].sum()
summary['Margin'] = summary['Profit']/summary['Subtotal Price']
# summary.to_excel("Summary2.xlsx")
summary

,Subtotal Cost,Subtotal Price,Profit,Margin
System,,,,
COMM SPARES,49716.21,51927.00,2210.79,0.04
ES,62614.96,98300.00,35685.04,0.36
FBB,34296.25,38798.00,4501.75,0.12
SPARES,155359.80,202748.00,47388.20,0.23
VSAT,1169072.97,1236132.00,67059.03,0.05


In [359]:
summary.to_excel('/Users/infowizard/Downloads/summary2.xlsx', index=False)

In [264]:
df['Subtotal Price LS'].sum()

1791012.0

In [255]:
# Report
print("Proposal report:\n")
print(f"Currency quoted in {quoted_currency}.")
print(f"Exchange rates are {currency}")
print(f"Cost escalation is {escalation * 100}%.")
print(f"Risk is {risk * 100}%.")
print(f"Default mark up is {recommended_mu*100}%.")
print("\n")
print(summary)
print("\n")
summary_to_report.sum()
print(str(total_margin * 100) + "%")

Proposal report:

Currency quoted in USD.
Exchange rates are {'SGD': 0.69, 'USD': 1.0, 'EUR': 1.0, 'GBP': 1.15, 'DKK': 0.1343, 'NOK': 0.0971, 'CNY': 0.1381, 'AUD': 0.64}
Cost escalation is 12.0%.
Risk is 5.0%.
Default mark up is 3.0%.


                      Subtotal Cost  Subtotal Price         Profit    Margin
System                                                                      
COMM SPARES            36062.396600         43453.0    7390.603400  0.170083
CRANE                  16698.522105         18810.0    2111.477895  0.112253
ENGINEERING SERVICES  169587.557474        261500.0  110905.600421  0.424113
GMDSS                 165430.116098        172308.0    6877.883902  0.039916
NAV                   179237.657229        222429.0   43191.342771  0.194180
OPERATION SUPPORT      28296.273183         32124.0    3827.726817  0.119155
SPARES                 46454.461642         59600.0   13145.538358  0.220563
UHF                   617229.840640        910969.0  293739.159360  0.

In [349]:
# Write commercial proposal
commercial_df = df[['No', 'Sub-No', 'Description', 'Qty', 'Unit', 'Unit Price LS', 'Subtotal Price LS', 'Scope',
                    'Format']]
commercial_df = commercial_df.rename(columns={"No" : "#", "Sub-No" : "SN", 'Unit Price LS' : 'Unit Price',
                                                'Subtotal Price LS' : 'Subtotal Price'})
# commercial_df.to_excel("Commercial " + "J12273 MODEC - BM-C-33 FPSO - COM R0.xlsx", index=False)
commercial_df.to_excel("/Users/infowizard/Downloads/J12272 MODEC - BM-C-33 FPSO - VSAT R0.xlsx", index=False)
# commercial_df.to_csv("commercial.csv")

In [256]:
filename

'09 J12273 MODEC - BM-C-33 FPSO - ENGINEERING SERVICES R0.xlsx'

In [ ]:
# Write technical Proposal
technical_df = df[['No', 'Sub-No', 'Description', 'Qty', 'Unit', 'Scope', 'Category']]
technical_df = technical_df.rename(columns={"No" : "#", "Sub-No" : "SN"})
technical_df.to_excel("Technical " + filename, index=False)

In [ ]:
# Write PDF output file.
# df = df.convert_dtypes()
df.to_excel("Report " + filename, index=False)